In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-clean-unbalanced/Dataset_clean_unbalanced.csv
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2012_to_2014.csv
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2009_to_2011.csv
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2005_to_2007.csv
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accident_coords_update.ipynb
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/ukTrafficAADF.csv
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/Areas.shp
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/Local_Authority_Districts_Dec_2016.geojson


In [ ]:
#Objectif : mis en place d'un model prédictif sur la sévérité d'un accident

# donnée de sortie (Y) : Accident_Severity

# données sans intérêt : Accident_Index , Local_Authority_(District) , Local_Authority_(Highway) , 1st_Road_Number , Junction_Detail ,
#                        2nd_Road_Number , Year , Location_Easting_OSGR , Location_Northing_OSGR , Longitude , Latitude , 
#                        LSOA_of_Accident_Location

# données d'intérêt (X) : Police_Force , Number_of_Vehicles , Number_of_Casualties , Date , Day_of_Week , Time ,
#                         1st_Road_Class , Road_Type , Speed_limit , Pedestrian_Crossing-Human_Control , Pedestrian_Crossing-Physical_Facilities ,
#                         Light_Conditions , Weather_Conditions , Road_Surface_Conditions , Special_Conditions_at_Site , Carriageway_Hazards ,
#                         Urban_or_Rural_Area , Did_Police_Officer_Attend_Scene_of_Accident , Junction_Control

'''
TODO :  dataset déséquilivré, faire 1ère analyse pour comparer avant/après ré-équilibrage
'''

In [2]:
df1=pd.read_csv('/kaggle/input/dataset-clean-unbalanced/Dataset_clean_unbalanced.csv')
df1.drop(columns =['Unnamed: 0'], inplace = True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1452458 entries, 0 to 1452457
Data columns (total 21 columns):
 #   Column                                       Non-Null Count    Dtype  
---  ------                                       --------------    -----  
 0   Police_Force                                 1452458 non-null  int64  
 1   Number_of_Vehicles                           1452458 non-null  int64  
 2   Number_of_Casualties                         1452458 non-null  int64  
 3   Day_of_Week                                  1452458 non-null  int64  
 4   1st_Road_Class                               1452458 non-null  int64  
 5   Road_Type                                    1452458 non-null  int64  
 6   Speed_limit                                  1452458 non-null  int64  
 7   Junction_Control                             1452458 non-null  int64  
 8   2nd_Road_Class                               1452458 non-null  int64  
 9   Pedestrian_Crossing-Human_Control            1

In [ ]:
'''
TEPMS DE TRAITMENT :
KNN = 2h10
DT = qqsec
RF = 6min
SVM = pas fini après 10h
'''

In [3]:
df1['Mois']=df1['Mois'].astype(int)
df1['Heure']=df1['Heure'].astype(int)

In [4]:
import matplotlib as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [5]:
X = df1.loc[:, df1.columns!='Severity']
Y = df1['Severity']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [7]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
classifierKNN = KNeighborsClassifier()
classifierKNN.fit(X_train, y_train)
y_predKNN = classifierKNN.predict(X_test)
#accuracy
accKNN = accuracy_score(y_test, y_predKNN)
print("accuracy_score =",accKNN)


accuracy_score = 0.8292551946353084


In [8]:
#confusion matrix
cmKNN = confusion_matrix(y_test, y_predKNN)
matrice_confKNN=pd.DataFrame(cmKNN,index=['1','2','3'],columns=['1','2','3'])
matrice_confKNN

,1,2,3
1,55,397,3308
2,178,2684,36628
3,438,8651,238153


In [9]:
#report
print(classification_report(y_test,y_predKNN))

              precision    recall  f1-score   support

           1       0.08      0.01      0.02      3760
           2       0.23      0.07      0.10     39490
           3       0.86      0.96      0.91    247242

    accuracy                           0.83    290492
   macro avg       0.39      0.35      0.35    290492
weighted avg       0.76      0.83      0.79    290492



In [9]:
#DT
from sklearn import tree
clfDT = tree.DecisionTreeClassifier(criterion='entropy',random_state=0)
clfDT = clfDT.fit(X_train, y_train)
y_predDT = clfDT.predict(X_test)
accDT=accuracy_score(y_test, y_predDT)
accDT

0.7505817716150531

In [12]:
cmDT = confusion_matrix(y_test, y_predDT)
matrice_confDT=pd.DataFrame(cmDT,index=['1','2','3'],columns=['1','2','3'])
matrice_confDT

,1,2,3
1,306,879,2575
2,1034,9213,29243
3,3573,35150,208519


In [13]:
print(classification_report(y_test,y_predDT))

              precision    recall  f1-score   support

           1       0.06      0.08      0.07      3760
           2       0.20      0.23      0.22     39490
           3       0.87      0.84      0.86    247242

    accuracy                           0.75    290492
   macro avg       0.38      0.39      0.38    290492
weighted avg       0.77      0.75      0.76    290492



In [15]:
#RF
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100,criterion='entropy',random_state=0) #n_estimators:nbr d'arbres(default=100)
rfc.fit(X_train, y_train)
rfc_pred_test = rfc.predict(X_test)
accRF=accuracy_score(y_test, rfc_pred_test)
accRF

0.8447255001858915

In [16]:
cmRF = confusion_matrix(y_test, rfc_pred_test)
matrice_confRF=pd.DataFrame(cmRF,index=['1','2','3'],columns=['1','2','3'])
matrice_confRF

,1,2,3
1,120,275,3365
2,76,3195,36219
3,183,4988,242071


In [17]:
print(classification_report(y_test,rfc_pred_test))

              precision    recall  f1-score   support

           1       0.32      0.03      0.06      3760
           2       0.38      0.08      0.13     39490
           3       0.86      0.98      0.92    247242

    accuracy                           0.84    290492
   macro avg       0.52      0.36      0.37    290492
weighted avg       0.79      0.84      0.80    290492



In [ ]:
'''
#normal SVM
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)
y_predSVC = svclassifier.predict(X_test)
print(accuracy_score(y_test,y_predSVC))
print(confusion_matrix(y_test,y_predSVC))
print(classification_report(y_test,y_predSVC))
'''